In [1]:
import os

def getSubFolders(folderPath):
    subFolders = []
    for root, dirs, files in os.walk(folderPath):
        for fn in files:            
            if fn.endswith('.asc') and 'Plate' in fn:
                pth = os.path.join(root, fn)
                if root in subFolders:
                    continue
                subFolders.append(root)
    return subFolders



def getMatch(string1, string2):
    from difflib import SequenceMatcher
    match = SequenceMatcher(None, string1, string2).find_longest_match(0, len(string1), 0, len(string2))
    return string1[match.a: match.a + match.size] 

def getCommonName(lf):
    answers = []
    for i, fo in enumerate(lf):
        for j, foj in enumerate(lf):        
            if getMatch(fo,foj)=='':
                continue
            answers.append(getMatch(fo,foj))

    l = [[x,answers.count(x)] for x in set(answers)]
    v = -1
    big = []
    for m in l:
        if m[1]>v:
            v= m[1]
            big = m
    common_name = big[0]
    return common_name



In [2]:
def getPlateNum(fn, s0='Plate'):
    p0 = fn.find(s0)    
    return fn[p0:len(fn)]

def generateWellName():
    list1 = map(chr, range(65, 73))
    list2 = range(1,13,1)
    list2 = map(str, list2)  
    well_name = []
    for l in list1:
        for n in list2:
            well_name.append(l+n)
    return well_name

def getValuesfromPlate(fn, values_in_plate):
    import numpy as np
    import re  
    import os
    f = open(fn, 'r')    
    well_names = generateWellName() 
    for line in f:        
        line_content = re.split(r'\t+',line)
        name = line_content[0]
        if name in well_names:            
            od = float(line_content[1])
            try: 
                fluo1 = float(line_content[2])
            except:
                fluo1 = -1         
            try:
                fluo2 = float(line_content[3])
            except:
                fluo2 = -1
            if name not in values_in_plate.keys():
                values_in_plate[name]= []
            values_in_plate[name].append([od,fluo1,fluo2])
    f.close()


def alphanum_key(s):
    import re
    convert = lambda text: int(text) if text.isdigit() else text 
    return [convert(c) for c in re.split('([0-9]+)', s)]


def plateToFile(values_in_plate, plate_name, time_plate,output_path):
    import csv
    import numpy as np
    output_folder = os.path.join(output_path,plate_name)
    if os.path.isdir(output_folder):
        import shutil
        shutil.rmtree(output_folder)        
    if not os.path.isdir(output_folder):
        os.mkdir(output_folder)    
    keys = values_in_plate.keys()
    keys.sort(key=alphanum_key)        
    value_matrix = dict()
    out_names = ['OD','GFP_100','GFP_50','GFP_extra']
    head_line = ['']

    for well in keys:
        head_line.append(well)
        values = values_in_plate[well]
        npvalues = np.asarray(values)
        [r,c] = npvalues.shape
        for col in range(c):        
            v_m = npvalues[:,col]
            v_m = v_m.reshape(-1,1)
            if col not in value_matrix.keys():
                value_matrix[col] = v_m
            else:
                value_matrix[col] = np.concatenate((value_matrix[col], v_m), axis = 1)

    t = time_plate
    for col in value_matrix.keys():
        out_name = out_names[int(col)]
        fn = os.path.join(output_folder,plate_name+'_'+str(out_name)+'.csv')        
        with open(fn, mode='w') as data_file:
            data_writer = csv.writer(data_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)        
            data_writer.writerow(head_line)
            mat = value_matrix[col]
            [r, c] = mat.shape
            for i in range(r):
                a = [t[i]]
                a.extend(mat[i,:].tolist())
                data_writer.writerow(a)
    return output_folder
                
def runPreprocessing(folderPath, extension='.asc'):
    import csv
    import os
    import numpy as np
    
    content = 'Plate'
    fns =[]
    num_plate = []
       
    for root, dirs, files in os.walk(folderPath):
        for fn in files:            
            if fn.endswith(extension) and content in fn:
                pth = os.path.join(root, fn)
                fns.append(os.path.abspath(pth))                
                if getPlateNum(fn, content) not in num_plate:
                    num_plate.append(getPlateNum(fn, content))

    files_order_by_plate = dict()
    for val in num_plate:        
        my_list = filter(lambda x: val in x, fns)
        files_order_by_plate[val] = my_list

    plates_reader = dict()
    
    outl = []    
    time_plate = {k: [] for k in files_order_by_plate.keys()}
    for plate in sorted(files_order_by_plate.keys()):         
        list_fns_plate = files_order_by_plate[plate]        
        values_in_plate = dict()        
        list_fns_plate.sort(key=lambda x: os.path.getmtime(x))        
        for fn in list_fns_plate:       
            #print fn            
            try:
                mtime = os.path.getmtime(fn)
            except OSError:
                mtime = 0
            time_plate[plate].append(mtime)            
            getValuesfromPlate(fn, values_in_plate)
        plates_reader[plate] = values_in_plate 
        # to be changed according to the table
        plate_name = plate[0:plate.find('.')] 
        #od_file = os.pe
        outputPath = os.path.split(fn)[0]                
        out = plateToFile(values_in_plate, plate_name, time_plate[plate],outputPath)        
        outl.append(out)
    return outl

In [3]:
def parsePlateName(pName):
    import re
    ds = re.findall(r'\d+', pName)
    return int(ds[0])


def getFnwithNum(l, num):
    for m in l:
        name = os.path.split(m)[1]        
        if num == parsePlateName(name):            
            return m


In [4]:
def mkdir_p(path):
    import errno    
    import os
    try:
        os.makedirs(path)
    except OSError as exc:  # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise
            


In [5]:
def smooth(a,WSZ=5):
    # a: NumPy 1-D array containing the data to be smoothed
    # WSZ: smoothing window size needs, which must be odd number,
    # as in the original MATLAB implementation
    out0 = np.convolve(a,np.ones(WSZ,dtype=int),'valid')/WSZ    
    r = np.arange(1,WSZ-1,2)

    start = np.cumsum(a[:WSZ-1])[::2]/r
    #print start,out0
    stop = (np.cumsum(a[:-WSZ:-1])[::2]/r)[::-1]
    return np.concatenate((  start , out0, stop  ))

def digitalOnly(s):
    import string
    al=string.maketrans('','')
    nodigs=al.translate(al, string.digits)
    return s.translate(al, nodigs)

def getLinearRegressionCoef(X,y):
    import numpy as np
    from sklearn.linear_model import LinearRegression
    reg = LinearRegression().fit(X, y)
    reg.score(X, y)    
    return reg.coef_

def computeDerivetive_od(t, sOD, wsz = 5):
    from numpy import isnan
    coef_OD = np.zeros(len(sOD))
    t_mean = np.zeros(len(sOD))
    j = 0
    nan_found = False
    for i in range(len(sOD)):
        t_wsz = t[i-wsz/2:i+wsz/2+1]        
        sOD_wsz = np.expand_dims(np.log(sOD[i-wsz/2:i+wsz/2+1]),axis=1)    
        if len(sOD_wsz)!= wsz:
            coef_OD[i] = 0
        else:                
            if len(t_wsz)==len(sOD_wsz):                
                where_are_NaNs = isnan(sOD_wsz)
                if any(where_are_NaNs):
                    if isnan(np.mean(sOD_wsz[~where_are_NaNs])):
                        sOD_wsz[where_are_NaNs] = 0
                    else:
                        sOD_wsz[where_are_NaNs] = np.mean(sOD_wsz[~where_are_NaNs])
                    print "nan Found in array, replaced by mean value of the rest which is : ",sOD_wsz[where_are_NaNs]

                coef_OD[i]=getLinearRegressionCoef(t_wsz,sOD_wsz)
                t_mean[i] = np.mean(t_wsz)
    
    dt = t_mean[wsz/2:len(t_mean)-wsz/2]
    dt_pad = np.pad(dt,[wsz/2,wsz/2],'reflect',reflect_type='odd')
    
    return coef_OD, dt_pad

def computeDerivetive(t, sOD, sGFP, wsz = 5):# od is firstly log computed
    from numpy import isnan
    coef_OD = np.zeros(len(sOD))
    t_mean = np.zeros(len(sOD))
    j = 0
    nan_found = False
    for i in range(len(sOD)):
        t_wsz = t[i-wsz/2:i+wsz/2+1]        
        sOD_wsz = np.expand_dims(np.log(sOD[i-wsz/2:i+wsz/2+1]),axis=1)    
        if len(sOD_wsz)!= wsz:
            coef_OD[i] = 0
        else:                
            if len(t_wsz)==len(sOD_wsz):                    
                where_are_NaNs = isnan(sOD_wsz)
                if any(where_are_NaNs):
                    if isnan(np.mean(sOD_wsz[~where_are_NaNs])):
                        sOD_wsz[where_are_NaNs] = 0
                    else:
                        sOD_wsz[where_are_NaNs] = np.mean(sOD_wsz[~where_are_NaNs])
                    print "nan Found in array, replaced by mean value of the rest which is : ",sOD_wsz[where_are_NaNs]
                coef_OD[i]=getLinearRegressionCoef(t_wsz,sOD_wsz)
                t_mean[i] = np.mean(t_wsz)

    coef_GFP = np.zeros(len(sGFP))
    for i in range(len(sGFP)):
        t_wsz = t[i-wsz/2:i+wsz/2+1]
        sGFP_wsz = np.expand_dims(sGFP[i-wsz/2:i+wsz/2+1],axis=1)
        if len(sGFP_wsz)!= wsz:
            coef_GFP[i] = 0
        else:                
            if len(t_wsz)==len(sGFP_wsz):
                coef_GFP[i]=getLinearRegressionCoef(t_wsz,sGFP_wsz)


    dGFP = coef_GFP
    dOD = coef_OD
    dt = t_mean[wsz/2:len(t_mean)-wsz/2]
    dt_pad = np.pad(dt,[wsz/2,wsz/2],'reflect',reflect_type='odd')
    dGFPdt_divide_OD = dGFP/sOD    
    return dGFPdt_divide_OD, coef_OD, coef_GFP, dt_pad

def getTimepoints(coef_OD,sOD_log,dt_pad,start_ind):
    coef_OD_fromStart_ind = coef_OD[start_ind:]
    ind = np.argmax(coef_OD_fromStart_ind) + start_ind        
    max_v = coef_OD[ind]
    time_max_coefod = dt_pad[ind]        
    left_side_exp, right_side_exp,small_t,tan = draw_tangent(dt_pad,sOD_log,ind)
    time_left = dt_pad[left_side_exp]
    time_right = dt_pad[right_side_exp]  
    return time_max_coefod,time_left,time_right,left_side_exp,right_side_exp,small_t,tan

def generateWellName():
    list1 = map(chr, range(65, 73))
    list2 = range(1,13,1)
    list2 = map(str, list2)  
    well_name = []
    for l in list1:
        for n in list2:
            well_name.append(l+n)
    return well_name

well_name = generateWellName()
print well_name


def getPostiveStart(sOD):
    start_positive = 0
    for o in range(start_positive, len(sOD)):
        if sOD[o] <= 0:
            start_positive = o + 1
    if start_positive >= len(sOD):
        start_positive = -1
    return start_positive


def getStationaryTime(sOD):
    sOD = np.asarray(sOD)
    sta_index = len(sOD)-1
    last_value = sOD[sta_index]
    for i in range(len(sOD)-2,0,-1):
        if (abs(last_value-sOD[i])/last_value)>0.05:
            sta_index = i
            print "stationary phase found"
            break
    return sta_index


def draw_tangent(x1,y1,ind,e=0.1,plot=False):
    from scipy import interpolate
    import numpy as np
    import matplotlib.pyplot as plt
    x = np.squeeze(x1)
    y = np.squeeze(y1)
    # interpolate the data with a spline
    spl = interpolate.splrep(x,y)
    a = x[ind]
    fa = interpolate.splev(a,spl,der=0)     # f(a)
    fprime = interpolate.splev(a,spl,der=1) # f'(a)
    left = 0
    for i in range(ind,0,-1):        
        a_i = x[i]
        y_i = y[i]        
        tan_i = fa+fprime*(a_i-a) # tangent                
        if abs(tan_i-y_i) > abs(y[ind]*e):
            left = i
            break    
    right = len(x) - 1
    for i in range(ind+1,len(x)):        
        a_i = x[i]
        y_i = y[i]
        tan_i = fa+fprime*(a_i-a) # tangent        
        if abs(tan_i-y_i) > abs(y[ind]*e):
            right = i
            break
    small_t = np.linspace(x[left],x[right],10)
    tan = fa+fprime*(small_t-a) # tangent    
    if plot:
        plt.plot(a,fa,'om',small_t,tan,'--r')
        plt.plot(x,y)
    return left, right,small_t,tan

['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10', 'E11', 'E12', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F12', 'G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'G10', 'G11', 'G12', 'H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9', 'H10', 'H11', 'H12']


In [6]:
folder_input = '/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song'
lf = getSubFolders(folder_input)
folder_list = []
filter_out = ['Plate11','Plate12']
for subFolder in lf:
    folders = runPreprocessing(subFolder)
    l_n = []
    for f_n in folders:        
        if os.path.split(f_n)[1] in filter_out:            
            continue
        l_n.append(f_n)
    folder_list.extend(l_n)    
folder_list

['/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 5-7 + controls TS008 TS071/15 Feb 2018/Plate1',
 '/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 5-7 + controls TS008 TS071/15 Feb 2018/Plate10',
 '/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 5-7 + controls TS008 TS071/15 Feb 2018/Plate2',
 '/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 5-7 + controls TS008 TS071/15 Feb 2018/Plate3',
 '/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 5-7 + controls TS008 TS071/15 Feb 2018/Plate4',
 '/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 5-7 + controls TS008 TS071/15 Feb 2018/Plate5',
 '/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 5-7 + controls TS008 TS071/15 Feb 2018/

In [7]:
def getStats(m_OD, time, outFolder):
    import numpy as np
    import os
    preName = os.path.split(outFolder)[1]
    outName_Stat = os.path.join(outFolder, preName +'_stats.csv')
    figure_folder_path = os.path.join(outFolder,'FIG')
    mkdir_p(figure_folder_path)
    
    detectedColumn = range(96)
    names_array = map(str,range(96))
    filtered_name = []
    t = time.reshape(-1,1)
    t_k = (t[1:len(t)] - t[0:len(t)-1])/3600
    t_k = np.cumsum(t_k)
    t_k = np.insert(t_k,0,0)
    t_k = t_k.reshape(-1,1)

    growth_rate = dict()
    lag_phase = dict()
    blank_od = 0.04
    pre_max_v = 0.7
    import matplotlib.pyplot as plt
    import numpy as np
    t = t_k
    for l in range(m_OD[:,detectedColumn].shape[1]):
        fn = well_name[detectedColumn[l]]
        out_figer_Name = os.path.join(figure_folder_path, preName +'-'+ fn)
        
        if len(filtered_name):
            if fn not in filtered_name:
                continue
        od = m_OD[:,detectedColumn][:,l]
        od_array = od - np.min(blank_od)
        od_array = np.squeeze(np.asarray(od_array))
        sOD = smooth(od_array)    
        sOD_log = np.log(sOD)
        
        #---compute lagphase using pre-max-v ---#
        coef_OD, dt_pad = computeDerivetive_od(t, sOD)
        ind_max_dODdt = np.argmax(coef_OD)
        OD_exp = sOD[ind_max_dODdt]
        time_max_dODdt = dt_pad[ind_max_dODdt]
        #t_exp - t0 - ln(OD_exp/OD_t0)/mu_max
        start_positive = getPostiveStart(sOD)
        if start_positive > 0:
            print fn,'found negative od ', start_positive    
        lag_phase_pre = time_max_dODdt - np.log(OD_exp/sOD[start_positive])/pre_max_v
        m = t < lag_phase_pre
        start_ind = np.sum(m)

        #---detect the early death pattern---#
        early_death = False
        if sOD[0] > sOD[len(sOD)-1]:
            early_death = True
            #print fn, 'Early death pattern detected'
        if early_death:
            sOD_reverse = sOD[::-1]
            sOD = sOD_reverse
            coef_OD, dt_pad = computeDerivetive_od(t, sOD)
            
            ind_max_dODdt = np.argmax(coef_OD)
            od_max_coef = coef_OD[ind_max_dODdt]
            OD_exp = sOD[ind_max_dODdt]
            time_max_dODdt = dt_pad[ind_max_dODdt]
            # t_exp - t0 - ln(OD_exp/OD_t0)/mu_max
            start_positive = getPostiveStart(sOD)
            if start_positive > 0:
                print fn,'found negative od ', start_positive  
            
            lag_phase_reversed = time_max_dODdt - np.log(OD_exp/sOD[start_positive])/od_max_coef
            # this is an estimated lag_phase
            lag_phase_pre = t[len(t)-1]-lag_phase_reversed
            lag_phase[fn] = np.squeeze(lag_phase_pre[0])
            #print type(lag_phase[fn])

            x = t
            y = sOD[::-1]
            plt.title(names_array[l] + '-'+fn)
            plt.plot(x,y,label='OD-Early-Death-Pattern')    
            plt.axvline(x=lag_phase[fn],linestyle=':',color='purple')    
            plt.xlabel('time')
            plt.legend()
            plt.savefig(out_figer_Name+'-Early-Death.png')
            plt.close()
            growth_rate[fn] = 0    
            
            plt.title(names_array[l] + '-'+fn)
            plt.plot(x,y,label='OD-Early-Death-Pattern')    
            plt.axvline(x=lag_phase[fn],linestyle=':',color='purple')    
            plt.xlabel('time')
            plt.legend()                
            growth_rate[fn] = 0    
            plt.savefig(out_figer_Name+'-Early-Death.png')
            plt.close()
            continue
        
        if sOD[len(sOD)-1] < 0.1:
            print fn, 'Flat line (no growth, no lagpahse)'
            x = t
            plt.title(names_array[l] + '-'+fn)
            plt.plot(x,sOD,label='OD-NoGrowth')    
            plt.xlabel('time')
            plt.ylim(top=0.3)
            plt.legend()
            plt.savefig(out_figer_Name+'-NoGrowth.png')
            growth_rate[fn] = 0
            lag_phase[fn] = 0
            continue

        sGFP = sOD
        dGFPdt_divide_OD, coef_OD, coef_GFP, dt_pad= computeDerivetive(t,sOD,sGFP)        
        time_max_od,time_left, time_right,left_side_exp,right_side_exp,small_t,tan = getTimepoints(coef_OD,sOD_log,dt_pad,start_ind)

        
        sta_index = getStationaryTime(sOD)
        t_sta = dt_pad[sta_index]
        
        ind = np.argmax(coef_OD)        
        max_v = coef_OD[ind]    
        growth_rate[fn] = max_v
        # Update lag_phase using growth rate in an estimated window
        lag_phase_pre = time_max_dODdt - np.log(OD_exp/sOD[start_positive])/max_v
        lag_phase[fn] = lag_phase_pre

        x = t
        y = sOD_log
        plt.plot(x,y,label='log(OD)')
        plt.plot(x,y,small_t,tan,'--r',label='log(OD)')
        plt.title(names_array[l] + '-'+fn)
        plt.xlabel("time")
        w = coef_OD
        plt.plot(x, w,label='dOD/dt/OD') # can c                
        #plt.axvline(x=time_left,linestyle=':',color='red')
        plt.axvline(x=time_max_od,linestyle=':',color='green')
        #plt.axvline(x=time_right,linestyle=':',color='red')
        plt.legend()
        plt.savefig(out_figer_Name+'-dODdtOD.png')
        print out_figer_Name+'-dODdtOD.png'
        #plt.show()
        plt.close()
        
        plt.plot(x,sOD,label='OD')    
        plt.axvline(x=time_left,linestyle=':',color='red')
        plt.axvline(x=time_max_od,linestyle=':',color='green')
        plt.axvline(x=t_sta,linestyle=':',color='yellow')
        plt.axvline(x=time_right,linestyle=':',color='red')
        plt.axvline(x=lag_phase[fn],linestyle=':',color='purple')    
        
        plt.xlabel('time')
        plt.title(names_array[l] + '-'+fn)        
        plt.legend()
        plt.savefig(out_figer_Name+'-OD.png')
        #plt.show()
        plt.close()
    
    
    import pandas as pd
    df = pd.DataFrame.from_dict([growth_rate,lag_phase])
    df = df.transpose()
    df2 = df.reindex(well_name)
    df2.columns = ['Growth Rate', 'Lag Phase']
    df2.to_csv(outName_Stat)


In [8]:
def getODfileFromFolder(folderPath):    
    for root, dirs, files in os.walk(folderPath):
        for fn in files:            
            if fn.endswith('.csv') and 'OD' in fn:
                pth = os.path.join(root, fn)
                break
    return pth

import numpy as np
m_data = dict()
time = dict()
for fn in folder_list:
    print fn
    od_file_path = getODfileFromFolder(fn)
    m_value= np.loadtxt(open(od_file_path, "rb"), delimiter=",", skiprows=1)
    out_name = 'RES'
    m_data[fn] = np.asmatrix(m_value[:,1:])
    time[fn] = m_value[:,0]
    getStats(m_data[fn],time[fn], fn)


/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 5-7 + controls TS008 TS071/15 Feb 2018/Plate1
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 5-7 + controls TS008 TS071/15 Feb 2018/Plate1/FIG/Plate1-A1-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 5-7 + controls TS008 TS071/15 Feb 2018/Plate1/FIG/Plate1-A2-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 5-7 + controls TS008 TS071/15 Feb 2018/Plate1/FIG/Plate1-A3-dODdtOD.png
A4 Flat line (no growth, no lagpahse)
A5 Flat line (no growth, no lagpahse)
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 5-7 + controls TS008 TS071/15 Feb 2018/Plate1/FIG/Plate1-A6-dODdtOD.png
stationary phase found
/home

stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 5-7 + controls TS008 TS071/15 Feb 2018/Plate1/FIG/Plate1-H5-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 5-7 + controls TS008 TS071/15 Feb 2018/Plate1/FIG/Plate1-H6-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 5-7 + controls TS008 TS071/15 Feb 2018/Plate1/FIG/Plate1-H7-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 5-7 + controls TS008 TS071/15 Feb 2018/Plate1/FIG/Plate1-H8-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 5-7 + controls TS008 TS071/15 Feb 2018/Plate1/FIG/Plate1-H9-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter

stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 5-7 + controls TS008 TS071/15 Feb 2018/Plate10/FIG/Plate10-F3-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 5-7 + controls TS008 TS071/15 Feb 2018/Plate10/FIG/Plate10-F4-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 5-7 + controls TS008 TS071/15 Feb 2018/Plate10/FIG/Plate10-F5-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 5-7 + controls TS008 TS071/15 Feb 2018/Plate10/FIG/Plate10-F6-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 5-7 + controls TS008 TS071/15 Feb 2018/Plate10/FIG/Plate10-F7-dODdtOD.png
stationary phase found
/home/xsong/Desktop/An

stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 5-7 + controls TS008 TS071/15 Feb 2018/Plate2/FIG/Plate2-G2-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 5-7 + controls TS008 TS071/15 Feb 2018/Plate2/FIG/Plate2-G3-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 5-7 + controls TS008 TS071/15 Feb 2018/Plate2/FIG/Plate2-G4-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 5-7 + controls TS008 TS071/15 Feb 2018/Plate2/FIG/Plate2-G5-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 5-7 + controls TS008 TS071/15 Feb 2018/Plate2/FIG/Plate2-G6-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter

stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 5-7 + controls TS008 TS071/15 Feb 2018/Plate3/FIG/Plate3-F1-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 5-7 + controls TS008 TS071/15 Feb 2018/Plate3/FIG/Plate3-F2-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 5-7 + controls TS008 TS071/15 Feb 2018/Plate3/FIG/Plate3-F3-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 5-7 + controls TS008 TS071/15 Feb 2018/Plate3/FIG/Plate3-F4-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 5-7 + controls TS008 TS071/15 Feb 2018/Plate3/FIG/Plate3-F5-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter

...

stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 1-3 + controls TS008 TS071/1 Feb 2018/Plate6/FIG/Plate6-B5-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 1-3 + controls TS008 TS071/1 Feb 2018/Plate6/FIG/Plate6-C1-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 1-3 + controls TS008 TS071/1 Feb 2018/Plate6/FIG/Plate6-C2-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 1-3 + controls TS008 TS071/1 Feb 2018/Plate6/FIG/Plate6-C3-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 1-3 + controls TS008 TS071/1 Feb 2018/Plate6/FIG/Plate6-C4-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-deri

/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 1-3 + controls TS008 TS071/1 Feb 2018/Plate7
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 1-3 + controls TS008 TS071/1 Feb 2018/Plate7/FIG/Plate7-A1-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 1-3 + controls TS008 TS071/1 Feb 2018/Plate7/FIG/Plate7-A2-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 1-3 + controls TS008 TS071/1 Feb 2018/Plate7/FIG/Plate7-A3-dODdtOD.png
A4 Flat line (no growth, no lagpahse)
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 1-3 + controls TS008 TS071/1 Feb 2018/Plate7/FIG/Plate7-B1-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadi

stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 1-3 + controls TS008 TS071/1 Feb 2018/Plate7/FIG/Plate7-H5-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 1-3 + controls TS008 TS071/1 Feb 2018/Plate7/FIG/Plate7-H6-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 1-3 + controls TS008 TS071/1 Feb 2018/Plate7/FIG/Plate7-H7-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 1-3 + controls TS008 TS071/1 Feb 2018/Plate7/FIG/Plate7-H8-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 1-3 + controls TS008 TS071/1 Feb 2018/Plate7/FIG/Plate7-H9-dODdtOD.png
H12 Flat line (no growth, no lagpahse)
/home/xsong/Desktop/A

/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 1-3 + controls TS008 TS071/1 Feb 2018/Plate8/FIG/Plate8-G3-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 1-3 + controls TS008 TS071/1 Feb 2018/Plate8/FIG/Plate8-G4-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 1-3 + controls TS008 TS071/1 Feb 2018/Plate8/FIG/Plate8-G5-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 1-3 + controls TS008 TS071/1 Feb 2018/Plate8/FIG/Plate8-G6-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 1-3 + controls TS008 TS071/1 Feb 2018/Plate8/FIG/Plate8-G7-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/a

stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 1-3 + controls TS008 TS071/1 Feb 2018/Plate9/FIG/Plate9-E2-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 1-3 + controls TS008 TS071/1 Feb 2018/Plate9/FIG/Plate9-E3-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 1-3 + controls TS008 TS071/1 Feb 2018/Plate9/FIG/Plate9-E4-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 1-3 + controls TS008 TS071/1 Feb 2018/Plate9/FIG/Plate9-E5-dODdtOD.png
stationary phase found
/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/asc_data/Mutant Data - Song/Mutant 1-3 + controls TS008 TS071/1 Feb 2018/Plate9/FIG/Plate9-E6-dODdtOD.png
E7 Flat line (no growth, no lagpahse)
stationary phase found